# Chapter 5 구조적 API 기본연산

In [1]:
from pyspark.sql import SparkSession # SparkSession: 스파크 코드를 실행하기 위한 진입점

spark = SparkSession \
    .builder \
    .appName("Python Spark SQL basic example") \
    .config("spark.some.config.option", "some-value") \
    .getOrCreate()

In [3]:
df = spark.read.format('json').load("/Users/taewoong/Documents/coding/Spark_practice/data/flight-data/json/2015-summary.json")

In [5]:
df.printSchema()

root
 |-- DEST_COUNTRY_NAME: string (nullable = true)
 |-- ORIGIN_COUNTRY_NAME: string (nullable = true)
 |-- count: long (nullable = true)



## 5.1 스키마

스키마는 DataFrame의 컬럼명과 데이터 타입을 정의합니다. 

In [6]:
df.schema

StructType(List(StructField(DEST_COUNTRY_NAME,StringType,true),StructField(ORIGIN_COUNTRY_NAME,StringType,true),StructField(count,LongType,true)))

스키마는 여러 개의 StructField 타입 필드로 구성된 StructType 객체이다.컬럼과 관련되 메타데이터를 지정 할 수도 있다. 메타데이터는 해당 컬럼과 관련된 정보이며 스프크의 머신러닝 라이브러리에서 사용한다.

In [7]:
from pyspark.sql.types import StructField, StructType, StringType, LongType

In [8]:
myManualSchema = StructType([
    StructField('DEST_COUNTRY_NAME',StringType(),True),
    StructField('ORIGIN_COUNTRY_NAME',StructType(),True),
    StructField("count",LongType(),False,metadata = {'hello':"world"})
])

In [9]:
df = spark.read.format("json").schema(myManualSchema).load('/Users/taewoong/Documents/coding/Spark_practice/data/flight-data/json/2015-summary.json')

In [12]:
df.printSchema()

root
 |-- DEST_COUNTRY_NAME: string (nullable = true)
 |-- ORIGIN_COUNTRY_NAME: struct (nullable = true)
 |-- count: long (nullable = true)



## 5.2 컬럼과 표현식

사용자는 **표현식**으로 DataFrame의 컬럼을 선택, 조작, 제거할 수 있다.<br>
DataFrame을 통하지 않으면 외부에서 컬럼에 접근할 수 없다. 컬럼 내용을 수정하려면 반드시 DataFrame의 스파크 트랜스포메이션을 사용해야 한다.

In [14]:
from pyspark.sql.functions import col, column

col("someColumnsName")
column("someColumnsName")

Column<b'someColumnsName'>

컬럼명은 컬럼명을 카탈로그에 저장된 정보와 비교하기 전까지 미확인 상태로 남습니다. 

* 명시적 컬럼 참조
    * DataFrame의 컬럼은 col 메서드로 참조한다. col 메서드는 조인 시 유용하다

**표현식**은 DataFramen 레코드의 여러 값에 대한 트렌스표메이션 집합을 의미한다. <br>
표현식은 expr함수로 가장 간단히 사용할 수 있다.

* 표현식으로 컬럼 표현
    * 컬럼은 단지 표현식일 뿐이다.
    * 컬럼과 컬럼의 트랜스포메이션은 파싱된 표련식과 동일한 논리적 실행 계획으로 컴파일 된다.

In [15]:
from pyspark.sql.functions import expr

expr("(((someCol + 5) * 200) - 6) < otherCol")

Column<b'((((someCol + 5) * 200) - 6) < otherCol)'>

DataFrame 코드나 SQL로 표현식을 작성할 수 있으며 동일한 성능을 발휘한다.


* DataFrame 컬럼에 접근하기 
    - printSchema 메서드로 DataFrame의 전체 컬럼 정보를 확인할 수 있다. 하지만 프로그래밍 방식으로 컬럼에 접근할 때는 DataFrame의 columns 속성을 사용한다

In [17]:
spark.read.format('json').load('/Users/taewoong/Documents/coding/Spark_practice/data/flight-data/json/2015-summary.json').columns

['DEST_COUNTRY_NAME', 'ORIGIN_COUNTRY_NAME', 'count']

## 5.3 레코드와 로우

* 스파크에서 DataFrame의 각 로우는 하나의 레코드이다. 
* 스파크는 레코드를 Row 객체로 표현한다. 
* 스파크는 값을 생성하기 위해 컬럼 표혀식으로 Row 객체를 다룬다. 
* Row 객체는 내부 바이트 배열을 가진다. 
* 바이트 배열 인터페이스는 오직 컬럼 표현식으로만 다룰 수 있으므로 사용자에게 절대 노출되지 않는다.

In [18]:
from pyspark.sql import Row

myRow = Row('Hello',None,1,False)

In [22]:
myRow[0]
myRow[2]

1

## 5.4 DataFrame의 트랜스포메이션

* 로우나 컬럼 추가
* 로우나 컬럼 제거
* 로우를 컬럼으로 변환하거나, 그 반대로 변환
* 컬럼값을 기준으로 로우 순서 변경


![](./IMG_E4C8B6943031-1.jpeg)

In [24]:
df = spark.read.format('json').load('./data/flight-data/json/2015-summary.json')

In [25]:
df.createOrReplaceTempView('dfTable')

In [27]:
df.show()

+--------------------+-------------------+-----+
|   DEST_COUNTRY_NAME|ORIGIN_COUNTRY_NAME|count|
+--------------------+-------------------+-----+
|       United States|            Romania|   15|
|       United States|            Croatia|    1|
|       United States|            Ireland|  344|
|               Egypt|      United States|   15|
|       United States|              India|   62|
|       United States|          Singapore|    1|
|       United States|            Grenada|   62|
|          Costa Rica|      United States|  588|
|             Senegal|      United States|   40|
|             Moldova|      United States|    1|
|       United States|       Sint Maarten|  325|
|       United States|   Marshall Islands|   39|
|              Guyana|      United States|   64|
|               Malta|      United States|    1|
|            Anguilla|      United States|   41|
|             Bolivia|      United States|   30|
|       United States|           Paraguay|    6|
|             Algeri

In [28]:
myManualSchema = StructType([
    StructField('some',StringType(),True),
    StructField('col',StringType(),True),
    StructField('names',LongType(),False),
    
])

In [29]:
myRow = Row('hello',None,1)
myDf = spark.createDataFrame([myRow],myManualSchema)
myDf.show()

+-----+----+-----+
| some| col|names|
+-----+----+-----+
|hello|null|    1|
+-----+----+-----+



* 컬럼이나 표현식을 사용하는 select 메서드
* 문자열 표현식을 사용하는 selectExpr 메서드
* 메서드로 사용할 수 없는 org.apache.spark.sql.functions 패키지에 포함된 다양한 함수


이 세 가지 유형의 메서드로 DataFrame을 다룰 때 필요한 대부분의 트랜스포메이션 작업을 해결할 수 있다

* select와 selectExpr 
        
        select와 selectExpr 메서드를 사용하면 데이터 테이블에 SQL을 실행하는 것처럼 DataFrame에서도 SQL을 사용할 수 있다

In [30]:
df.select("DEST_COUNTRY_NAME").show(2)

+-----------------+
|DEST_COUNTRY_NAME|
+-----------------+
|    United States|
|    United States|
+-----------------+
only showing top 2 rows



In [32]:
df.select("DEST_COUNTRY_NAME","ORIGIN_COUNTRY_NAME").show(2)

+-----------------+-------------------+
|DEST_COUNTRY_NAME|ORIGIN_COUNTRY_NAME|
+-----------------+-------------------+
|    United States|            Romania|
|    United States|            Croatia|
+-----------------+-------------------+
only showing top 2 rows



In [33]:
df.select(expr("DEST_COUNTRY_NAME"),
         col("DEST_COUNTRY_NAME"),
         column("DEST_COUNTRY_NAME")).show()

+--------------------+--------------------+--------------------+
|   DEST_COUNTRY_NAME|   DEST_COUNTRY_NAME|   DEST_COUNTRY_NAME|
+--------------------+--------------------+--------------------+
|       United States|       United States|       United States|
|       United States|       United States|       United States|
|       United States|       United States|       United States|
|               Egypt|               Egypt|               Egypt|
|       United States|       United States|       United States|
|       United States|       United States|       United States|
|       United States|       United States|       United States|
|          Costa Rica|          Costa Rica|          Costa Rica|
|             Senegal|             Senegal|             Senegal|
|             Moldova|             Moldova|             Moldova|
|       United States|       United States|       United States|
|       United States|       United States|       United States|
|              Guyana|   

In [34]:
df.select(col("DEST_COUNTRY_NAME"),"DEST_COUNTRY_NAME")

DataFrame[DEST_COUNTRY_NAME: string, DEST_COUNTRY_NAME: string]

In [35]:
df.select(expr("DEST_COUNTRY_NAME AS destination" )).show()

+--------------------+
|         destination|
+--------------------+
|       United States|
|       United States|
|       United States|
|               Egypt|
|       United States|
|       United States|
|       United States|
|          Costa Rica|
|             Senegal|
|             Moldova|
|       United States|
|       United States|
|              Guyana|
|               Malta|
|            Anguilla|
|             Bolivia|
|       United States|
|             Algeria|
|Turks and Caicos ...|
|       United States|
+--------------------+
only showing top 20 rows



In [38]:
df.select(expr("DEST_COUNTRY_NAME AS destination" ).alias("DEST_COUNTRY_NAME")).show(2)

+-----------------+
|DEST_COUNTRY_NAME|
+-----------------+
|    United States|
|    United States|
+-----------------+
only showing top 2 rows



select 메서드에 expr함수를 사용하는 패턴 -> **selectExpr**

In [39]:
df.selectExpr(
"*",
"(DEST_COUNTRY_NAME = ORIGIN_COUNTRY_NAME) as withCountry").show()

+--------------------+-------------------+-----+-----------+
|   DEST_COUNTRY_NAME|ORIGIN_COUNTRY_NAME|count|withCountry|
+--------------------+-------------------+-----+-----------+
|       United States|            Romania|   15|      false|
|       United States|            Croatia|    1|      false|
|       United States|            Ireland|  344|      false|
|               Egypt|      United States|   15|      false|
|       United States|              India|   62|      false|
|       United States|          Singapore|    1|      false|
|       United States|            Grenada|   62|      false|
|          Costa Rica|      United States|  588|      false|
|             Senegal|      United States|   40|      false|
|             Moldova|      United States|    1|      false|
|       United States|       Sint Maarten|  325|      false|
|       United States|   Marshall Islands|   39|      false|
|              Guyana|      United States|   64|      false|
|               Malta|  

* 집계 함수

In [40]:
df.selectExpr("avg(count)","count(distinct(DEST_COUNTRY_NAME))").show(2)

+-----------+---------------------------------+
| avg(count)|count(DISTINCT DEST_COUNTRY_NAME)|
+-----------+---------------------------------+
|1770.765625|                              132|
+-----------+---------------------------------+



때로는 새로운 컬럼이 아닌 명시적인 값을 스파크에 전달해야 한다. 명시적인 값은 상숫값일수 있고, 추후 비교에 사용할 무언가가 될 수도 있다. 이때 리터럴을 사용한다

In [41]:
from pyspark.sql.functions import lit

df.select(expr("*"),lit(1).alias('one')).show(2)

+-----------------+-------------------+-----+---+
|DEST_COUNTRY_NAME|ORIGIN_COUNTRY_NAME|count|one|
+-----------------+-------------------+-----+---+
|    United States|            Romania|   15|  1|
|    United States|            Croatia|    1|  1|
+-----------------+-------------------+-----+---+
only showing top 2 rows



DataFrame에 신규 컬럼을 추가하는 공식 : DataFrame의 withColum 메서드를 사용

In [45]:
df.withColumn("numberOne",lit(1)).show(2)

+-----------------+-------------------+-----+---------+
|DEST_COUNTRY_NAME|ORIGIN_COUNTRY_NAME|count|numberOne|
+-----------------+-------------------+-----+---------+
|    United States|            Romania|   15|        1|
|    United States|            Croatia|    1|        1|
+-----------------+-------------------+-----+---------+
only showing top 2 rows



In [46]:
df.withColumn("withinCountry",expr("ORIGIN_COUNTRY_NAME==DEST_COUNTRY_NAME")).show(2)

+-----------------+-------------------+-----+-------------+
|DEST_COUNTRY_NAME|ORIGIN_COUNTRY_NAME|count|withinCountry|
+-----------------+-------------------+-----+-------------+
|    United States|            Romania|   15|        false|
|    United States|            Croatia|    1|        false|
+-----------------+-------------------+-----+-------------+
only showing top 2 rows



withColumn 메서드는 두 개의 인수를 사용한다. 하나는 컬럼명, 다른 하나는 값을 생성할 표현식이다

withColumn 메서드 대신 withColumnRenamed 메서드로 컬럼명을 변경할 수도 있다.

In [47]:
df.withColumnRenamed("DEST_COUNTRY_NAME","dest").show(2)

+-------------+-------------------+-----+
|         dest|ORIGIN_COUNTRY_NAME|count|
+-------------+-------------------+-----+
|United States|            Romania|   15|
|United States|            Croatia|    1|
+-------------+-------------------+-----+
only showing top 2 rows



공백이나 하이픈 같은 문자는 컬럼명에 사용할 수 없다. 예약 뮨자를 컬럼명에 사용하려면 백틱 문자를 이용해 이스케이핑 해야한다.

* 컬럼 제거하기

In [48]:
df.drop("ORIGIN_COUNTRY_NAME").columns

['DEST_COUNTRY_NAME', 'count']

* 컬럼의 데이터 타입 변경하기

cast 메서드 사용

In [50]:
df.withColumn("count2",col("count").cast("string")).show(2)

+-----------------+-------------------+-----+------+
|DEST_COUNTRY_NAME|ORIGIN_COUNTRY_NAME|count|count2|
+-----------------+-------------------+-----+------+
|    United States|            Romania|   15|    15|
|    United States|            Croatia|    1|     1|
+-----------------+-------------------+-----+------+
only showing top 2 rows



* 로우 필터링

로우를 필터링 하려면 참과 거짓을 판별하는 표현식을 만들어야 한다. DataFrame의 where 메서드나 filter 메서드로 필터링할 수 있다

In [51]:
df.filter(col("count")<2).show(2)

+-----------------+-------------------+-----+
|DEST_COUNTRY_NAME|ORIGIN_COUNTRY_NAME|count|
+-----------------+-------------------+-----+
|    United States|            Croatia|    1|
|    United States|          Singapore|    1|
+-----------------+-------------------+-----+
only showing top 2 rows



In [52]:
df.where("count<2").show(2)

+-----------------+-------------------+-----+
|DEST_COUNTRY_NAME|ORIGIN_COUNTRY_NAME|count|
+-----------------+-------------------+-----+
|    United States|            Croatia|    1|
|    United States|          Singapore|    1|
+-----------------+-------------------+-----+
only showing top 2 rows



In [57]:
df.where("count<2").where(col("ORIGIN_COUNTRY_NAME") != "Croatia").show(2)

+-----------------+-------------------+-----+
|DEST_COUNTRY_NAME|ORIGIN_COUNTRY_NAME|count|
+-----------------+-------------------+-----+
|    United States|          Singapore|    1|
|          Moldova|      United States|    1|
+-----------------+-------------------+-----+
only showing top 2 rows



* 고유한 로우 얻기

distinct 메서드를 사용해서 고윳값을 찾을 수 있다.


In [60]:
df.select("ORIGIN_COUNTRY_NAME","DEST_COUNTRY_NAME").distinct().count()

256

* 무작위 샘플 만들기

sample 메서드를 사용한다. DataFrame에서 표본 데이터 추출 비율을 지정할 수 있다. 복원 추출이나 비복원 추출의 사용여부를 지정할 수 있다


In [61]:
seed = 5
withReplacement = False
fraction = 0.5
df.sample(withReplacement,fraction,seed).count()

138

In [62]:
df.sample(withReplacement,fraction,seed).show()

+--------------------+--------------------+-----+
|   DEST_COUNTRY_NAME| ORIGIN_COUNTRY_NAME|count|
+--------------------+--------------------+-----+
|       United States|             Romania|   15|
|       United States|             Croatia|    1|
|       United States|               India|   62|
|       United States|           Singapore|    1|
|       United States|             Grenada|   62|
|             Senegal|       United States|   40|
|             Moldova|       United States|    1|
|       United States|    Marshall Islands|   39|
|              Guyana|       United States|   64|
|             Bolivia|       United States|   30|
|       United States|            Paraguay|    6|
|       United States|           Gibraltar|    1|
|Saint Vincent and...|       United States|    1|
|               Italy|       United States|  382|
|       United States|Federated States ...|   69|
|       United States|              Russia|  161|
|       United States|         Netherlands|  660|


* 임의 분할하기

random split은 원본 DataFrame을 임의 크기로 '분할'할때 유용하게 사용된다. randomSplit 메서드는 임의성을 가지도록 설계되었으므로 시드값을 반드시 설정해야 한다. 총합이 1이 되도록 각 DataFrame의 비율을 지정하지 않으면 예제와 같은 비율로 지정된다.

In [63]:
dataFrames = df.randomSplit([0.25,0.75],seed)

In [65]:
dataFrames[0].count() > dataFrames[1].count()

False

In [68]:
print(dataFrames[0].count())
print(dataFrames[1].count())

71
185


* 로우 합치기와 추가하기

DataFrame은 불변성을 가진다. 그러므로 DataFrame에 레코드를 추가하는 작업은 DataFrame을 변경하는 작업이기 때문에 출가능 하다. DataFrame에 레코드 추가하려면 원본 DataFrame을 새로운 DataFrame과 **통합**해야한다. 통합은 두 개의 DataFrame을 단순히 결합하는 행위이다. 통합하려는 두 개의 DataFrame은 반드시 동이한 스키마와 컬럼 수를 가져야 한다.

In [72]:
from pyspark.sql import Row

schema = df.schema

newRows = [
    Row("New Country", "Other Country", 5),
    Row("New Country 2", "Other Country 3", 1)
]

In [73]:
parallelizedRows = spark.sparkContext.parallelize(newRows)

In [76]:
newDF = spark.createDataFrame(parallelizedRows,schema)

In [77]:
df.union(newDF).where("count = 1").where(col("ORIGIN_COUNTRY_NAME") != "United States").show()

+-----------------+-------------------+-----+
|DEST_COUNTRY_NAME|ORIGIN_COUNTRY_NAME|count|
+-----------------+-------------------+-----+
|    United States|            Croatia|    1|
|    United States|          Singapore|    1|
|    United States|          Gibraltar|    1|
|    United States|             Cyprus|    1|
|    United States|            Estonia|    1|
|    United States|          Lithuania|    1|
|    United States|           Bulgaria|    1|
|    United States|            Georgia|    1|
|    United States|            Bahrain|    1|
|    United States|   Papua New Guinea|    1|
|    United States|         Montenegro|    1|
|    United States|            Namibia|    1|
|    New Country 2|    Other Country 3|    1|
+-----------------+-------------------+-----+



* 로우 정렬하기 

sort와 orderBy 메서드를 사용해 DataFrame의 최댓값 혹은 최솟값이 상단에 위치하도록 정렬할 수 있다.

In [78]:
df.sort("count").show(5)

+--------------------+-------------------+-----+
|   DEST_COUNTRY_NAME|ORIGIN_COUNTRY_NAME|count|
+--------------------+-------------------+-----+
|               Malta|      United States|    1|
|Saint Vincent and...|      United States|    1|
|       United States|            Croatia|    1|
|       United States|          Gibraltar|    1|
|       United States|          Singapore|    1|
+--------------------+-------------------+-----+
only showing top 5 rows



In [79]:
df.orderBy(col("count"),col("DEST_COUNTRY_NAME")).show(3)

+-----------------+-------------------+-----+
|DEST_COUNTRY_NAME|ORIGIN_COUNTRY_NAME|count|
+-----------------+-------------------+-----+
|     Burkina Faso|      United States|    1|
|    Cote d'Ivoire|      United States|    1|
|           Cyprus|      United States|    1|
+-----------------+-------------------+-----+
only showing top 3 rows



In [80]:
from pyspark.sql.functions import desc, asc

In [81]:
df.orderBy(expr("count desc")).show(2)

+-----------------+-------------------+-----+
|DEST_COUNTRY_NAME|ORIGIN_COUNTRY_NAME|count|
+-----------------+-------------------+-----+
|          Moldova|      United States|    1|
|    United States|            Croatia|    1|
+-----------------+-------------------+-----+
only showing top 2 rows



In [82]:
df.orderBy(col("count").desc(),col("DEST_COUNTRY_NAME").asc()).show(2)

+-----------------+-------------------+------+
|DEST_COUNTRY_NAME|ORIGIN_COUNTRY_NAME| count|
+-----------------+-------------------+------+
|    United States|      United States|370002|
|    United States|             Canada|  8483|
+-----------------+-------------------+------+
only showing top 2 rows



* 로우 수 제한하기

DataFrame에서 추출할 로우 수를 제한해야 할 때가 있다. limit 메서드를 사용해 추출할 로우 수를 제한한다.

In [83]:
df.limit(5).show()

+-----------------+-------------------+-----+
|DEST_COUNTRY_NAME|ORIGIN_COUNTRY_NAME|count|
+-----------------+-------------------+-----+
|    United States|            Romania|   15|
|    United States|            Croatia|    1|
|    United States|            Ireland|  344|
|            Egypt|      United States|   15|
|    United States|              India|   62|
+-----------------+-------------------+-----+



* repartition과 coalesce

자주 필터링하는 컬럼을 기준으로 데이터를 분할한다.
이를 통해 파티션닝 스키마와 파티션 수를 포함해 클러스터 전반의 물리적 데이터 구성을 제어할 수 있다.<br>

repartition 메서드를 호출하면 무조건 전체 데이터를 셔플한다. 향후에 사용할 파티션 수가 현재 파티션 수보다 많거나 컬럼을 기준으로 파티션을 만드는 경우에만 사용해야 한다.

In [87]:
df.repartition(5).show()

+--------------------+-------------------+-----+
|   DEST_COUNTRY_NAME|ORIGIN_COUNTRY_NAME|count|
+--------------------+-------------------+-----+
|       United States|            Denmark|  152|
|       United States|         Martinique|   43|
|       United States|        Saint Lucia|  136|
|             Ireland|      United States|  335|
|         South Korea|      United States| 1048|
|       United States|              Italy|  438|
|       United States|             Greece|   23|
|Bonaire, Sint Eus...|      United States|   58|
|              France|      United States|  935|
|       United States|             Cyprus|    1|
|       United States|         Montenegro|    1|
|       United States|            Austria|   63|
|           Australia|      United States|  329|
|       United States|        New Zealand|   74|
|       United States|           Suriname|   34|
|       United States|           Malaysia|    3|
|       United States|             Guyana|   63|
|              Taiwa

In [89]:
df.repartition(col("DEST_COUNTRY_NAME")).show()

+--------------------+-------------------+-----+
|   DEST_COUNTRY_NAME|ORIGIN_COUNTRY_NAME|count|
+--------------------+-------------------+-----+
|            Anguilla|      United States|   41|
|              Russia|      United States|  176|
|            Paraguay|      United States|   60|
|             Senegal|      United States|   40|
|              Sweden|      United States|  118|
|            Kiribati|      United States|   26|
|              Guyana|      United States|   64|
|         Philippines|      United States|  134|
|            Djibouti|      United States|    1|
|            Malaysia|      United States|    2|
|           Singapore|      United States|    3|
|                Fiji|      United States|   24|
|              Turkey|      United States|  138|
|                Iraq|      United States|    1|
|             Germany|      United States| 1468|
|              Jordan|      United States|   44|
|               Palau|      United States|   30|
|Turks and Caicos ..

In [ ]:
coalesce 메서드는 전체 데이터를 셔플하지 않고 파티션을 병합하려는 경우에 사용한다

In [91]:
df.repartition(5,col("DEST_COUNTRY_NAME")).coalesce(2).show()

+--------------------+-------------------+-----+
|   DEST_COUNTRY_NAME|ORIGIN_COUNTRY_NAME|count|
+--------------------+-------------------+-----+
|             Moldova|      United States|    1|
|             Bolivia|      United States|   30|
|             Algeria|      United States|    4|
|Turks and Caicos ...|      United States|  230|
|            Pakistan|      United States|   12|
|    Marshall Islands|      United States|   42|
|            Suriname|      United States|    1|
|              Panama|      United States|  510|
|         New Zealand|      United States|  111|
|             Liberia|      United States|    2|
|             Ireland|      United States|  335|
|              Zambia|      United States|    1|
|            Malaysia|      United States|    2|
|               Japan|      United States| 1548|
|    French Polynesia|      United States|   43|
|           Singapore|      United States|    3|
|             Denmark|      United States|  153|
|               Spai

* 드라이버로 로우 데이터 수집하기

스파크는 드라이버에서 클러스터 상태 정보를 유지한다. 로컬 환경에서 데이터를 다루려면 드라이버로 데이터를 수집해야한다


In [92]:
collectDF =df.limit(10)
collectDF.take(5) # take는 정수형 값을 인수로 사용한다


[Row(DEST_COUNTRY_NAME='United States', ORIGIN_COUNTRY_NAME='Romania', count=15),
 Row(DEST_COUNTRY_NAME='United States', ORIGIN_COUNTRY_NAME='Croatia', count=1),
 Row(DEST_COUNTRY_NAME='United States', ORIGIN_COUNTRY_NAME='Ireland', count=344),
 Row(DEST_COUNTRY_NAME='Egypt', ORIGIN_COUNTRY_NAME='United States', count=15),
 Row(DEST_COUNTRY_NAME='United States', ORIGIN_COUNTRY_NAME='India', count=62)]

In [93]:
collectDF.show()

+-----------------+-------------------+-----+
|DEST_COUNTRY_NAME|ORIGIN_COUNTRY_NAME|count|
+-----------------+-------------------+-----+
|    United States|            Romania|   15|
|    United States|            Croatia|    1|
|    United States|            Ireland|  344|
|            Egypt|      United States|   15|
|    United States|              India|   62|
|    United States|          Singapore|    1|
|    United States|            Grenada|   62|
|       Costa Rica|      United States|  588|
|          Senegal|      United States|   40|
|          Moldova|      United States|    1|
+-----------------+-------------------+-----+



In [94]:
collectDF.show(5,False)

+-----------------+-------------------+-----+
|DEST_COUNTRY_NAME|ORIGIN_COUNTRY_NAME|count|
+-----------------+-------------------+-----+
|United States    |Romania            |15   |
|United States    |Croatia            |1    |
|United States    |Ireland            |344  |
|Egypt            |United States      |15   |
|United States    |India              |62   |
+-----------------+-------------------+-----+
only showing top 5 rows



In [95]:
collectDF.collect()

[Row(DEST_COUNTRY_NAME='United States', ORIGIN_COUNTRY_NAME='Romania', count=15),
 Row(DEST_COUNTRY_NAME='United States', ORIGIN_COUNTRY_NAME='Croatia', count=1),
 Row(DEST_COUNTRY_NAME='United States', ORIGIN_COUNTRY_NAME='Ireland', count=344),
 Row(DEST_COUNTRY_NAME='Egypt', ORIGIN_COUNTRY_NAME='United States', count=15),
 Row(DEST_COUNTRY_NAME='United States', ORIGIN_COUNTRY_NAME='India', count=62),
 Row(DEST_COUNTRY_NAME='United States', ORIGIN_COUNTRY_NAME='Singapore', count=1),
 Row(DEST_COUNTRY_NAME='United States', ORIGIN_COUNTRY_NAME='Grenada', count=62),
 Row(DEST_COUNTRY_NAME='Costa Rica', ORIGIN_COUNTRY_NAME='United States', count=588),
 Row(DEST_COUNTRY_NAME='Senegal', ORIGIN_COUNTRY_NAME='United States', count=40),
 Row(DEST_COUNTRY_NAME='Moldova', ORIGIN_COUNTRY_NAME='United States', count=1)]

전체 데이터셋에 대한 반복처리를 위해 드라이버로 로우를 모으는 또 다른 방법이 있다.
toLocalIterator 메서드는 이터레이터로 모든 파티션의 데이터를 드라이버에 전달한다.

In [96]:
collectDF.toLocalIterator()

<generator object _local_iterator_from_socket.<locals>.PyLocalIterable.__iter__ at 0x7fa0a57aa930>